<a href="https://colab.research.google.com/github/Allen001822480/Info7374SpringTeam5/blob/Final_Project/Final_dcgan_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pickle
text = np.load('text.npy')
image = np.load('image128.npy')
with open('word_index.pickle','rb') as v:
  word_index = pickle.load(v)

In [0]:
def scale_image(image):
  image = (image/ 255) * 2 - 1
  return image

In [0]:
image = scale_image(image)

In [0]:
import requests
file_url = "http://nlp.stanford.edu/data/glove.6B.zip"
tr = requests.get(file_url, stream=True)
with open("glove.6B.zip", "wb") as f:
    for chunk in tr.iter_content(chunk_size=1024):
        if chunk:
            f.write(chunk)
            
            
import zipfile
import os
def un_zip(file_name):
    """unzip zip file"""
    zip_file = zipfile.ZipFile(file_name)
    if os.path.isdir(file_name + "_files"):
        pass
    else:
        os.mkdir(file_name + "_files")
    for names in zip_file.namelist():
        zip_file.extract(names,file_name + "_files/")
    zip_file.close()
        
glove = un_zip("glove.6B.zip")

In [4]:
embeddings_index = {}
f = open('./glove.6B.zip_files/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [0]:
maxlen = 20
max_words = 5000
embedding_dim = 100
noise_dim = 10
embeddings_index = {}
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [6]:
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Embedding, LSTM, concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam, SGD

import matplotlib.pyplot as plt

#generator
noise_input = Input(shape=(noise_dim,))
noise_layer = Dense(embedding_dim)(noise_input)
sentence = Input(shape=(maxlen,))
sentence_layer = Embedding(output_dim=embedding_dim, input_dim=max_words, weights=[embedding_matrix], input_length=maxlen)(sentence)
sentence_layer = LSTM(embedding_dim)(sentence_layer)
merged_layer = concatenate([noise_layer, sentence_layer])
generator_layer = Dense(128 * 16 * 16, activation="relu")(merged_layer)
generator_layer = Reshape((16, 16, -1))(generator_layer)
generator_layer = UpSampling2D()(generator_layer)
generator_layer = Conv2D(128, kernel_size=3, padding="same")(generator_layer)
generator_layer = BatchNormalization(momentum=0.8)(generator_layer)
generator_layer = Activation("relu")(generator_layer)
generator_layer = UpSampling2D()(generator_layer)
generator_layer = Conv2D(64, kernel_size=3, padding="same")(generator_layer)
generator_layer = BatchNormalization(momentum=0.8)(generator_layer)
generator_layer = Activation("relu")(generator_layer)
generator_layer = UpSampling2D()(generator_layer)
generator_layer = Conv2D(32, kernel_size=3, padding="same")(generator_layer)
generator_layer = BatchNormalization(momentum=0.8)(generator_layer)
generator_layer = Activation("relu")(generator_layer)
generator_layer = Conv2D(3, kernel_size=3, padding="same")(generator_layer)
generator_output = Activation("tanh")(generator_layer)
generator = Model([noise_input,sentence], generator_output)
generator.summary() 

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 20, 100)      500000      input_2[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 100)          1100        input_1[0][0]                    
_____________________________________

In [7]:
image_input = Input(shape=(128,128,3))
image_layer = Conv2D(16, kernel_size=3, strides=2, padding="same")(image_input)
image_layer = LeakyReLU(alpha=0.2)(image_input)
image_layer = Dropout(0.25)(image_input)
image_layer = Conv2D(32, kernel_size=3, strides=2, padding="same")(image_input)
image_layer = ZeroPadding2D(padding=((0,1),(0,1)))(image_layer)
image_layer = BatchNormalization(momentum=0.8)(image_layer)
image_layer = LeakyReLU(alpha=0.2)(image_layer)
image_layer = Dropout(0.25)(image_layer)
image_layer = Conv2D(64, kernel_size=3, strides=2, padding="same")(image_layer)
image_layer = BatchNormalization(momentum=0.8)(image_layer)
image_layer = LeakyReLU(alpha=0.2)(image_layer)
image_layer = Dropout(0.25)(image_layer)
image_layer = Conv2D(128, kernel_size=3, strides=1, padding="same")(image_layer)
image_layer = BatchNormalization(momentum=0.8)(image_layer)
image_layer = LeakyReLU(alpha=0.2)(image_layer)
image_layer = Dropout(0.25)(image_layer)
image_layer = Flatten()(image_layer)
image_layer = Dense(100)(image_layer)
text_input = Input(shape=(maxlen,))
text_embedd = Embedding(output_dim=embedding_dim, input_dim=max_words, weights=[embedding_matrix], input_length=maxlen)(text_input)
text_rnn = LSTM(embedding_dim)(text_embedd)
merged = concatenate([image_layer,text_rnn])
discriminator_layer = Activation('tanh')(merged)
discriminator_layer = Dense(1)(discriminator_layer)
validity = Activation('sigmoid')(discriminator_layer)
discriminator = Model([image_input,text_input],validity)
discriminator.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 64, 64, 32)   896         input_3[0][0]                    
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 65, 65, 32)   0           conv2d_6[0][0]                   
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 65, 65, 32)   128         zero_padding2d_1[0][0]      

In [8]:
optimizer = Adam(lr=0.0005)
generator.compile(loss='mean_squared_error', optimizer=optimizer)
discriminator.compile(loss='binary_crossentropy',optimizer=optimizer)
model_output = discriminator([generator.output,sentence])
combined_model = Model([noise_input,sentence],model_output)
discriminator.trainable = False
combined_model.compile(loss='binary_crossentropy',optimizer=optimizer)
combined_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 20, 100)      500000      input_2[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 100)          1100        input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

In [0]:
epoch = 5
batch_size = 16
snapshot_interval = 100
snapshot_dir_path = './dcgan/snapshot/dcgan_01/'
generator_model_path = './dcgan/dcgan_01_generator.h5'
discriminator_model_path = './dcgan/dcgan_01_discriminator.h5'

In [10]:
print(text.shape)
print(image.shape)

(81890, 20)
(8189, 128, 128, 3)


In [0]:
import os,math
from PIL import Image

if not os.path.isdir(snapshot_dir_path):
  os.makedirs(snapshot_dir_path)
  
def combine_normalized_images(generated_images):
  num = generated_images.shape[0]
  width = int(math.sqrt(num))
  height = int(math.ceil(float(num) / width))
  shape = generated_images.shape[1:]
  image = np.zeros((height * shape[0], width * shape[1], shape[2]),
                     dtype=generated_images.dtype)
  for index, img in enumerate(generated_images):
    i = int(index / width)
    j = index % width
    image[i * shape[0]:(i + 1) * shape[0], j * shape[1]:(j + 1) * shape[1], :] = img
  return image
def img_from_normalized_img(normalized_img):
  image = normalized_img * 127.5 + 127.5
  return Image.fromarray(image.astype(np.uint8))
def save_snapshots(generated_images, snapshot_dir_path, epoch, batch_index):
  image = combine_normalized_images(generated_images)
  img_from_normalized_img(image).save(os.path.join(snapshot_dir_path, str(epoch) + "-" + str(batch_index) + ".png"))

In [0]:
if os.path.isfile(generator_model_path):
    generator.load_weights(generator_model_path)
if os.path.isfile(discriminator_model_path):
    discriminator.load_weights(discriminator_model_path)

In [0]:
import shutil
shutil.rmtree('./dcgan')

In [16]:
for i in range(epoch):
  batch_count = int(image.shape[0]/batch_size)
  for batch_index in range(batch_count):
    text_batch_real = text[batch_index * batch_size*10:(batch_index + 1) * batch_size*10,:]
    image_batch_real = image[batch_index * batch_size:(batch_index + 1) * batch_size,:,:,:]
    image_batch_real = np.tile(image_batch_real,(10,1,1,1))
    index1 = np.random.randint(low=0, high=text.shape[0], size=batch_size*10)
    text_batch_wrong = text[index1,:]
    index2 = np.random.randint(low=0, high=image.shape[0], size=batch_size)
    image_batch_wrong = image[index2,:,:,:]
    image_batch_wrong = np.tile(image_batch_wrong,(10,1,1,1))
    noise = np.zeros((batch_size*10, noise_dim))
    for index in range(batch_size*10):
      noise[index, :] = np.random.uniform(-1, 1, noise_dim)
    generated_images = generator.predict([noise,text_batch_real], verbose=0)
    if (epoch * batch_size + batch_index) % snapshot_interval == 0 and snapshot_dir_path is not None:
      save_snapshots(generated_images, snapshot_dir_path=snapshot_dir_path, epoch=i, batch_index=batch_index)
    #train the discriminator
    discriminator.trainable = True
    d_loss = discriminator.train_on_batch([np.concatenate((image_batch_real, generated_images,image_batch_wrong)),
                                                            np.concatenate((text_batch_real, text_batch_real, text_batch_wrong))],
                                                           np.array([1] * batch_size*10 + [0] * batch_size*10 + [0] * batch_size*10))
    print("Epoch %d batch %d d_loss : %f" % (i, batch_index, d_loss))
    #train the generator
    discriminator.trainable = False
    g_loss = combined_model.train_on_batch([noise,text_batch_real], np.array([1] * batch_size*10))
    print("Epoch %d batch %d g_loss : %f" % (i, batch_index, g_loss))
  generator.save_weights(generator_model_path, overwrite=True)
  discriminator.save_weights(discriminator_model_path, overwrite=True)



Epoch 0 batch 0 d_loss : 0.503644
Epoch 0 batch 0 g_loss : 0.669500
Epoch 0 batch 1 d_loss : 0.489570
Epoch 0 batch 1 g_loss : 0.656913
Epoch 0 batch 2 d_loss : 0.516297
Epoch 0 batch 2 g_loss : 0.646272
Epoch 0 batch 3 d_loss : 0.489053
Epoch 0 batch 3 g_loss : 0.642689
Epoch 0 batch 4 d_loss : 0.514349
Epoch 0 batch 4 g_loss : 0.637693
Epoch 0 batch 5 d_loss : 0.497098
Epoch 0 batch 5 g_loss : 0.627027
Epoch 0 batch 6 d_loss : 0.545087
Epoch 0 batch 6 g_loss : 0.615305
Epoch 0 batch 7 d_loss : 0.546942
Epoch 0 batch 7 g_loss : 0.614898
Epoch 0 batch 8 d_loss : 0.505830
Epoch 0 batch 8 g_loss : 0.615127
Epoch 0 batch 9 d_loss : 0.474159
Epoch 0 batch 9 g_loss : 0.617155
Epoch 0 batch 10 d_loss : 0.465588
Epoch 0 batch 10 g_loss : 0.627655
Epoch 0 batch 11 d_loss : 0.514962
Epoch 0 batch 11 g_loss : 0.639376
Epoch 0 batch 12 d_loss : 0.506115
Epoch 0 batch 12 g_loss : 0.652597
Epoch 0 batch 13 d_loss : 0.453092
Epoch 0 batch 13 g_loss : 0.668559
Epoch 0 batch 14 d_loss : 0.472950
Epoch

KeyboardInterrupt: ignored